In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import sys
import os
import json
import time
import re

In [18]:
headers0 = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:48.0) Gecko/20100101 Firefox/48.0',
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    'accept-language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'keep-alive': 'keep-alive',
    'cache-control': 'max-age=0',
}

In [4]:
URL = 'http://admining.hadoop1.corp.mail.ru/login/'
logn = 'a.tleubayev'
passwd = ''
client = requests.session()

client.get(URL)
csrftoken = client.cookies['csrftoken']

login_data = dict(username=logn, password=passwd,
                  csrfmiddlewaretoken=csrftoken, next='/')
r = client.post(URL, data=login_data, headers=dict(Referer=URL))

if len(r.content) < 5000:
    print('Неверно введен логин или пароль')
    exit(1)

In [5]:
url = 'http://admining.hadoop1.corp.mail.ru/api/load_tree/'
try:
    page = requests.get(url, timeout=10)
except requests.exceptions.Timeout:
    print('Ошибка подключения к admining.hadoop1.corp.mail.ru, проверьте подключение к VPN.')
    exit(1)
soup = BeautifulSoup(page.content, 'html.parser')
tree = json.loads(str(soup))

In [6]:
def dfs(tree, st):
    if 'test' in tree['a_attr']['title'].lower():
        print(st + tree['a_attr']['title'])
        return [], []
    
    nums = [tree['id']]
    text = [tree['text']]
    st += '    '
    if tree['children'] != None:
        for i in tree['children']:
            t = dfs(i, st)
            nums.extend(t[0])
            text.extend(t[1])
    return nums, text

In [7]:
nums = []
text = []
for i in tree:
#     print(i['a_attr']['title'])
    st = ''
    t = dfs(i, st)
    nums.extend(t[0])
    text.extend(t[1])

1207. test (Kirill)
1331. test_pavel
1342. Test (Denis)
1344. test (Timur)
1346. Test (ADIL)
1352. Test_Nastya


In [15]:
def getDataUrls(dataId):
    urls = []
    for categoryId in dataId:
        data = client.get('http://admining.hadoop1.corp.mail.ru/load_patterns/{}/'.format(categoryId)).content
        data_json = json.loads(data.decode())
        urls.extend([[i.split(' ')[0], categoryId] for i in data_json[u'patterns'].split('\n')])

    urls = np.array(urls)
    table = pd.DataFrame({'urls': urls.T[0], 'categoryId': urls.T[1]})
    table = pd.get_dummies(table, columns=['categoryId'])
    table = table.groupby('urls').sum()
    
    return table.index, table.values

In [22]:
%%time
dataUrls, dataCategories = make_data.getDataUrls(client, nums)

CPU times: user 10.6 s, sys: 1.14 s, total: 11.8 s
Wall time: 12min 44s


In [40]:
_time = time.time()
for idx, categoryId in enumerate(nums):
    data = client.get('http://admining.hadoop1.corp.mail.ru/load_patterns/{}/'.format(categoryId)).content
    data_json = json.loads(data.decode())
    
    f = open('data/categories/{}_urls.txt'.format(categoryId), 'w')
    for i in data_json[u'patterns'].split('\n'):
        f.write(i+'\n')
    f.close()
    sys.stdout.write('\r' + str(idx+1) + '/' + str(len(nums)) + '\t' + 'time: ' + str(time.time() - _time))

899/899	time: 718.19336295127873

In [2]:
class LoggingPool(ThreadPool):
    def apply_async(self, func, args=(), kwds={}, callback=None):
        return ThreadPool.apply_async(self, LogExceptions(func), args, kwds, callback)

In [71]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse

# def getText(domain):
#     " Cкачать текст с главной страницы домена domain "
#     try:    
#         page = requests.get(domain, headers=headers0, timeout=100)
#         soup = BeautifulSoup(page.content)

#         for script in soup(["script", "style"]):
#             script.extract()

#         text = soup.get_text()
#         lines = (line.strip() for line in text.splitlines())
#         chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#         text = '\n'.join(chunk for chunk in chunks if chunk)
        
#     except:
#         text = 'NOTEXT'
#     return text

def getText(domain):
    " Cкачать текст с главной страницы домена domain "
    try:    
        response = urlopen(domain)
        htmlBytes = response.read()
        soup = BeautifulSoup(htmlBytes)

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        
    except:
        text = 'NOTEXT'
    return text

In [4]:
import pymorphy2
import re

re_extract_words = re.compile('[^а-яёa-z0-9]')
re_repeat_spaces = re.compile('[ ]+')

morph = pymorphy2.MorphAnalyzer()

def morph_tokenizer(text):
    ext_text = re_extract_words.sub (u' ', text.lower())
    words = re_repeat_spaces.sub (u' ', ext_text)
    words = words.split (u' ')
    
    norms = [ morph.parse (w)[0].normal_form  for w in words  if (len (w) > 1) ]
    return norms

In [51]:
pool = LoggingPool(processes=64)
_time = time.time()

categoryId = 91
f = open('data/categories/{}_urls.txt'.format(categoryId))
lines = f.readlines()
f.close()

urls = [i.split(' ')[0].replace('\n', '') for i in lines]
# texts = pool.map(getText, urls)
# textsToken = pool.map(morph_tokenizer, texts)
# texts = [' '.join(i) for i in textsToken]

In [26]:
urls = ['cyberforum.ru', 'olx.kz', 'price.ru', 'online-pc.ru']

texts = pool.map(getText, urls)
textsToken = pool.map(morph_tokenizer, texts)
texts = [' '.join(i) for i in textsToken]

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /home/adil/.pyenv/versions/3.6.1/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [28]:
texts[-1]

'продажа компьютер по низкий цена от 9000р компьютерный магазин предлагать за 9000р 155000р купить компьютер офис сборка на заказ наш интернет магазин компьютер главный компания контакт гарантия доставка способ оплата не дозвониться оставить ваш телефон мы перезвонить ваш имя ваш телефон указать причинуникто не брать трубкупроблема соединениеммна быть удобно если вы презвонить выбрать менеджер оценить мы похвалить пожаловаться посоветовать читать отзыв каталогноутбукипланшетыкомпьютер игровой домашний офисный эконом стандарт комфорт премиум миниатюрный неттоп микро компьютер минь компьютер рабочий станция для создание дизайн для 3d графика для видеомонтаж для проектирование для работа на биржа выгодный комплект без компромисс стильный мощный моноблок сервер hp lenovo acer сборка на заказмониторыпринтёр мфусканерымультимедиасетевой оборудованиеустройство ручной вводаtv тюнерыибп сетевой фильтрыкорпусапрограммный обеспечениеоргтехникадополнительный компьютерамкомплектовать поиск разделыд

In [5]:
nums = ['Aktiv', 'Citilink']

In [72]:
i = 5
categ = data[0][i][0]
regul = data[0][i][1]
urls = data[1][i]
texts = pool.map(getText, urls)
textsToken = pool.map(morph_tokenizer, texts)
texts = [' '.join(i) for i in textsToken]

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /home/adil/.pyenv/versions/3.6.1/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [79]:
pool = LoggingPool(processes=64)
_time = time.time()
    
for idx, categoryId in enumerate(nums):
    f = open('myNewData/text{}.txt'.format(categoryId), 'w')
    data = np.load('myNewData/data{}.npy'.format(categoryId))
    for tt in range(data[0].shape[0]):
        categ = data[0][tt][0]
        regul = data[0][tt][1]
        if tt in data[1]:
            urls = data[1][tt]
            texts = pool.map(getText, urls)
            textsToken = pool.map(morph_tokenizer, texts)
            texts = [' '.join(i) for i in textsToken]
#             print(len(texts))

            for idx2, k in enumerate(texts):
                f.write(categ + ' ' + regul + ' ' + urls[idx2] + ' ' + k +'\n')
    f.close()
    sys.stdout.write('\r' + str(idx+1) + '/' + str(len(nums)) + '\t' + 'time: ' + str(time.time() - _time))

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /home/adil/.pyenv/versions/3.6.1/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


1/2	time: 1457.2975914478302

KeyboardInterrupt: 

In [6]:
# pool = LoggingPool(processes=64)
# _time = time.time()

# for idx, categoryId in enumerate(nums):
#     f = open('data/categories/{}_urls.txt'.format(categoryId))
#     lines = f.readlines()
#     f.close()

#     urls = [i.split(' ')[0].replace('\n', '') for i in lines]
#     texts = pool.map(getText, urls)
#     textsToken = pool.map(morph_tokenizer, texts)
#     texts = [' '.join(i) for i in textsToken]

#     f = open('data/categories/{}_text.txt'.format(categoryId), 'w')
#     for i in texts:
#         f.write(i+'\n')
#     f.close()
#     sys.stdout.write('\r' + str(idx+1) + '/' + str(len(nums)) + '\t' + 'time: ' + str(time.time() - _time))

In [97]:
sm

83383

In [41]:
def getText(domain):
    " Cкачать текст с главной страницы домена domain "
    try:    
#         page = requests.get('http://'+domain)#, headers=headers0, timeout=100)
        page = urllib.request.urlopen('http://'+domain)
#         response = http.request('GET', 'http://'+domain)
#         response
        soup = BeautifulSoup(page.read(), 'html5lib')

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()
#         lines = (line.strip() for line in text.splitlines())
#         chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#         text = '\n'.join(chunk for chunk in chunks if chunk)
        
    except:
        text = 'NOTEXT'
    return text

In [100]:
categoryId = 1458
_time = time.time()
f = open('data/categories/{}_urls.txt'.format(categoryId))
lines = f.readlines()
f.close()

urls = [i.split(' ')[0].replace('\n', '') for i in lines]
texts = pool.map(getText, urls)
textsToken = pool.map(morph_tokenizer, texts)
texts = [' '.join(i) for i in textsToken]

f = open('{}_text.txt'.format(categoryId), 'w')
for i in texts:
    f.write(i+'\n')
f.close()
print(time.time() - _time)

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /home/adil/.pyenv/versions/3.6.1/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


KeyboardInterrupt: 

In [14]:
categoryId = 1458
_time = time.time()
f = open('data/categories/{}_urls.txt'.format(categoryId))
lines = f.readlines()
f.close()

urls = [i.split(' ')[0].replace('\n', '') for i in lines]

In [42]:
pool = LoggingPool(processes=64)
_time = time.time()
texts = pool.map(getText, urls[:50])
# textsToken = pool.map(morph_tokenizer, texts)
# texts = [' '.join(i) for i in textsToken]
print(time.time() - _time)

131.08696937561035


In [ ]:
texts

In [20]:
import urllib3
import urllib

In [36]:
page = urllib.request.urlopen('http://'+urls[0])

In [44]:
print(nums)

[91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 138

In [2]:
nums = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 599, 605, 606, 607, 608, 609, 611, 610, 612, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1455, 1456, 1457, 1458, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753, 777, 836, 844, 845, 1494, 1495, 1496, 846, 1407, 1415, 1418, 1420, 1451, 1409, 1414, 1492, 1493, 849, 850, 887, 847, 1406, 1410, 1413, 1416, 1417, 1452, 1419, 1453, 1485, 1486, 848, 888, 1412, 1487, 1488, 1489, 1490, 890, 891, 1421, 1408, 889, 1411, 851, 853, 852, 857, 858, 859, 1146, 1147, 1148, 1338, 1339, 1340, 1341, 854, 855, 856, 1276, 860, 861, 862, 885, 886, 1238, 1239, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1285, 1278, 1279, 1280, 1281, 1284, 778, 779, 780, 781, 782, 783, 784, 785, 786, 800, 801, 787, 791, 790, 792, 793, 795, 796, 797, 802, 803, 804, 805, 1427, 1159, 1391, 808, 809, 811, 812, 1260, 813, 1392, 810, 866, 958, 1059, 1060, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1141, 1142, 1380, 1423, 1160, 1473, 788, 806, 807, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 1119, 1107, 1122, 1330, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 1108, 867, 868, 869, 1096, 1231, 1232, 1233, 1098, 1503, 870, 871, 872, 873, 874, 875, 1011, 1018, 1019, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1200, 1202, 1203, 1225, 1498, 876, 877, 878, 879, 881, 882, 883, 1364, 893, 894, 948, 949, 950, 912, 913, 946, 947, 1396, 914, 915, 943, 944, 945, 951, 952, 920, 921, 922, 923, 924, 925, 684, 686, 1230, 926, 1329, 927, 928, 930, 938, 940, 931, 932, 933, 934, 935, 1403, 1404, 1405, 1047, 1052, 1053, 1054, 1049, 1050, 1051, 1055, 1256, 1123, 1258, 1048, 1257, 1259, 960, 961, 972, 962, 963, 964, 965, 973, 974, 975, 976, 977, 978, 979, 966, 967, 968, 969, 971, 980, 982, 985, 880, 989, 986, 987, 988, 1502, 1001, 1002, 1004, 1005, 1006, 1007, 1131, 990, 991, 993, 994, 995, 992, 1128, 1129, 1130, 1033, 1057, 1058, 1034, 1056, 1038, 1039, 1043, 1044, 1035, 1040, 1042, 1045, 1036, 1135, 1218, 1426, 1467, 1037]

In [3]:
sm = 0
for idx, categoryId in enumerate(nums):
    f = open('data/categories/{}_urls.txt'.format(categoryId))
    lines = f.readlines()
    sm += len(lines)
    f.close()
sm

123795

In [4]:
sm = 0
for idx, categoryId in enumerate(nums):
    f = open('data/categories/{}_urls.txt'.format(categoryId))
    lines = f.readlines()
    sm += len(lines)
    f.close()
    if idx == 579:
        break
sm

83395